In [13]:
import rx_4n_2k.model as model
import rx_4n_2k.__init__ as init
import os
import h5py

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import pandas as pd
from tensorflow.keras.optimizers import Adam


In [38]:
k = 2
n = 2

learning_rate = 1e-2
n_epochs = 10

weight_path = 'rx_4n_2k\weights.h5'
input_data_path = r'received_iq.csv'
label_data_path = r'actual_labels.csv'

best_model_name_and_path = r'best_model.h5'

In [15]:
def load_model(load_weights=True, debug=False):
    m = model.create_model()
    if load_weights:
        loadWeights(m, debug=debug)
    return m

## Utility functions:


def loadWeights(model, filename=weight_path, debug=False):
    with h5py.File(filename, 'r') as f:
        # Every layer is an h5 group. Ignore non-groups (such as /0)
        for g in f:
            if isinstance(f[g], h5py.Group):
                group = f[g]
                layerName = group.attrs['Name']
                numVars = int(group.attrs['NumVars'])
                if debug:
                    print("layerName:", layerName)
                    print("    numVars:", numVars)
                # Find the layer index from its namevar
                layerIdx = layerNum(model, layerName)
                layer = model.layers[layerIdx]
                if debug:
                    print("    layerIdx=", layerIdx)
                # Every weight is an h5 dataset in the layer group. Read the weights 
                # into a list in the correct order
                weightList = [0]*numVars
                for d in group:
                    dataset = group[d]
                    varName = dataset.attrs['Name']
                    shp     = intList(dataset.attrs['Shape'])
                    weightNum = int(dataset.attrs['WeightNum'])
                    # Read the weight and put it into the right position in the list
                    if debug:
                        print("    varName:", varName)
                        print("        shp:", shp)
                        print("        weightNum:", weightNum)
                    weightList[weightNum] = tf.constant(dataset[()], shape=shp)
                # Assign the weights into the layer
                for w in range(numVars):
                    if debug:
                        print("Copying variable of shape:")
                        print(weightList[w].shape)
                    layer.variables[w].assign(weightList[w])
                    if debug:
                        print("Assignment successful.")
                        print("Set variable value:")
                        print(layer.variables[w])
                # Finalize layer state
                if hasattr(layer, 'finalize_state'):
                    layer.finalize_state()

def layerNum(model, layerName):
    # Returns the index to the layer
    layers = model.layers
    for i in range(len(layers)):
        if layerName==layers[i].name:
            return i
    print("")
    print("WEIGHT LOADING FAILED. MODEL DOES NOT CONTAIN LAYER WITH NAME: ", layerName)
    print("")
    return -1

def intList(myList): 
    # Converts a list of numbers into a list of ints.
    return list(map(int, myList))

In [29]:
def decimal_to_onehot_array(decimal_array,k):

  one_hot_array = np.zeros((len(decimal_array), 2**k), dtype=np.float32)

  # Use bitwise AND operation with powers of 2 to get individual bits
  for i, num in enumerate(decimal_array):
    one_hot_array[i,num]=1
    # for j in range(k):
    #   one_hot_array[i, k-j-1] = (num & (2**j)) != 0
  return one_hot_array


In [30]:
decoder = load_model(load_weights=True, debug=False)

# Getting data and setting up inputs and outputs to the model
df_inp = pd.read_csv(input_data_path,header =None)
df_out = pd.read_csv(label_data_path,header= None)
print(df_inp.head())
inp_stream = np.array([complex(i.replace('i','j')) for i in df_inp.iloc[:, 0]])
print(inp_stream[0:4])
inp_stream = np.concatenate((np.reshape(np.real(inp_stream),(-1,1)),np.reshape(np.imag(inp_stream),(-1,1))),axis=1)
inp_stream = np.reshape(inp_stream,(-1,2*n))  # convert to a one demensional array
out_stream = np.array([int(i) for i in df_out.iloc[:, 0]])
print(out_stream[0:4])
out_stream = decimal_to_onehot_array(out_stream,k)
print(out_stream[0:4])
# Spliting
train_x,val_x = np.split(inp_stream,[int(0.75 * len(inp_stream))])
train_y,val_y = np.split(out_stream,[int(0.75 * len(out_stream))])
print(inp_stream.shape)
print(out_stream.shape)
print(train_x.shape,train_y.shape)
print(val_x.shape,val_y.shape)

                                        0
0   -0.299422364232565-0.196514844617445i
1  -0.123069949070052+0.0469773601491152i
2   -0.331475405155046-0.231331290499908i
3  -0.136061816847937+0.0649977259307791i
4   -0.204147585407668+0.215834226385493i
[-0.29942236-0.19651484j -0.12306995+0.04697736j -0.33147541-0.23133129j
 -0.13606182+0.06499773j]
[1 1 3 1]
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
(838800, 4)
(838800, 4)
(629100, 4) (629100, 4)
(209700, 4) (209700, 4)


In [31]:
decoder.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 4)]               0         
                                                                 
 fc_3_ (Dense)               (None, 4)                 20        
                                                                 
 re_lu_9 (ReLU)              (None, 4)                 0         
                                                                 
 fc_4_ (Dense)               (None, 4)                 20        
                                                                 
 softmax_9 (Softmax)         (None, 4)                 0         
                                                                 
Total params: 40 (160.00 Byte)
Trainable params: 40 (160.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
# Train decoder
decoder.compile(optimizer=Adam(learning_rate=learning_rate),loss="binary_crossentropy",metrics = ['accuracy'])
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience =n_epochs//10)
mc = ModelCheckpoint(best_model_name_and_path,monitor='val_accuracy',mode='max',verbose=1,save_best_only=True)
history = decoder.fit(train_x,train_y,validation_data = (val_x,val_y),epochs = n_epochs,verbose=2)

Epoch 1/10
19660/19660 - 26s - loss: 0.0060 - accuracy: 0.9963 - val_loss: 2.1375e-08 - val_accuracy: 1.0000 - 26s/epoch - 1ms/step
Epoch 2/10
19660/19660 - 29s - loss: 1.1186e-05 - accuracy: 1.0000 - val_loss: 2.7479e-10 - val_accuracy: 1.0000 - 29s/epoch - 1ms/step
Epoch 3/10
19660/19660 - 26s - loss: 6.8074e-06 - accuracy: 1.0000 - val_loss: 3.6576e-08 - val_accuracy: 1.0000 - 26s/epoch - 1ms/step
Epoch 4/10
19660/19660 - 27s - loss: 1.4059e-05 - accuracy: 1.0000 - val_loss: 7.9800e-11 - val_accuracy: 1.0000 - 27s/epoch - 1ms/step
Epoch 5/10
19660/19660 - 27s - loss: 9.0138e-06 - accuracy: 1.0000 - val_loss: 2.5147e-08 - val_accuracy: 1.0000 - 27s/epoch - 1ms/step
Epoch 6/10
19660/19660 - 26s - loss: 1.3633e-05 - accuracy: 1.0000 - val_loss: 8.4987e-11 - val_accuracy: 1.0000 - 26s/epoch - 1ms/step
Epoch 7/10
19660/19660 - 27s - loss: 9.0925e-06 - accuracy: 1.0000 - val_loss: 4.4043e-09 - val_accuracy: 1.0000 - 27s/epoch - 1ms/step
Epoch 8/10
19660/19660 - 25s - loss: 2.0463e-05 - ac